<a href="https://colab.research.google.com/github/Geoffrey256/meta-learning-final-year/blob/main/reptileCopy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import random
import tensorflow as tf
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.datasets import ImageFolder
import shutil

import glob


from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
train_dir1 = "/content/drive/MyDrive/Datasets/Extracted/Pharmaceutical_and_Biomedical_Waste/Pharmaceutical and Biomedical Waste (PBW)"
train_dir = '/content/drive/MyDrive/Datasets/Extracted/organized_dataset/train'
val_dir = '/content/drive/MyDrive/Datasets/Extracted/organized_dataset/validation'
test_dir = '/content/drive/MyDrive/Datasets/Extracted/organized_dataset/test'

In [ ]:
# @title Step 2: Data Preprocessing and Augmentation

# Define data augmentation transformations
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Load and transform the images from the directories
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)


Found 2547 images belonging to 13 classes.
Found 849 images belonging to 13 classes.
Found 849 images belonging to 13 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define a CNN-based model architecture suitable for few-shot learning
class ReptileModel(tf.keras.Model):
    def __init__(self, num_classes=1):  # Adjust `num_classes` as needed
        super(ReptileModel, self).__init__()

        # Convolutional layers
        self.conv1 = layers.Conv2D(32, (3, 3), activation='relu')
        self.conv2 = layers.Conv2D(64, (3, 3), activation='relu')
        self.conv3 = layers.Conv2D(128, (3, 3), activation='relu')
        self.pool = layers.MaxPooling2D((2, 2))

        # Flatten and Dense layers
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(128, activation='relu')
        self.fc2 = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        return self.fc2(x)

# Initialize the model
model = ReptileModel(num_classes=13)  # Assuming 12 classes in your dataset


In [ ]:
import tensorflow as tf

# Define loss function and optimizer outside the training function
loss_fn = tf.keras.losses.CategoricalCrossentropy()  # Define the loss function
optimizer = tf.keras.optimizers.Adam()               # Define the optimizer

# Training function
def train_reptile_model(model, train_data, num_epochs=5, steps_per_epoch=50):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        for step in range(steps_per_epoch):
            X_batch, y_batch = next(train_data)  # Get the next batch

            with tf.GradientTape() as tape:
                predictions = model(X_batch, training=True)  # Make predictions
                loss = loss_fn(y_batch, predictions)         # Calculate loss

            # Compute gradients and update weights
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            # Calculate accuracy
            accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(predictions, axis=1), tf.argmax(y_batch, axis=1)), tf.float32))

            if step % 10 == 0:  # Print every 10 steps
                print(f"Step: {step}, Loss: {loss.numpy()}, Accuracy: {accuracy.numpy() * 100:.2f}%")

# # Example setup: calculate steps per epoch or set explicitly
# steps_per_epoch = train_data.samples // train_data.batch_size
# train_reptile_model(model, train_data, num_epochs=5, steps_per_epoch=steps_per_epoch)


In [ ]:
# Calculate steps per epoch based on your training data size
steps_per_epoch = train_data.samples // train_data.batch_size

# Call the training function
train_reptile_model(model, train_data, num_epochs=5, steps_per_epoch=steps_per_epoch)


Epoch 1/5
Step: 0, Loss: 2.586129903793335, Accuracy: 3.12%
Step: 10, Loss: 2.5671610832214355, Accuracy: 12.50%
Step: 20, Loss: 2.3435871601104736, Accuracy: 21.88%
Step: 30, Loss: 2.3425679206848145, Accuracy: 12.50%
Step: 40, Loss: 2.343461275100708, Accuracy: 18.75%
Step: 50, Loss: 1.8867439031600952, Accuracy: 34.38%
Step: 60, Loss: 1.9476189613342285, Accuracy: 28.12%
Step: 70, Loss: 2.068929672241211, Accuracy: 18.75%
Epoch 2/5
Step: 0, Loss: 1.922595500946045, Accuracy: 25.00%
Step: 10, Loss: 1.8436733484268188, Accuracy: 28.12%
Step: 20, Loss: 1.8790991306304932, Accuracy: 37.50%
Step: 30, Loss: 1.7931338548660278, Accuracy: 34.38%
Step: 40, Loss: 1.8741836547851562, Accuracy: 25.00%
Step: 50, Loss: 1.7944025993347168, Accuracy: 40.62%
Step: 60, Loss: 1.9460580348968506, Accuracy: 25.00%
Step: 70, Loss: 1.7937191724777222, Accuracy: 12.50%
Epoch 3/5
Step: 0, Loss: 1.760842204093933, Accuracy: 26.32%
Step: 10, Loss: 1.3647758960723877, Accuracy: 34.38%
Step: 20, Loss: 1.7741236